In [13]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [14]:
import sys
import os

# Get the absolute path to the parent directory containing "02504 Computer Vision"
parent_dir = os.path.abspath(os.path.join("../../..", "02504-Computer-Vision"))

# Add it to sys.path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from utility import Pi, Piinv, projectpoints, point_line_distance, projection_matrix

In [15]:
ransac = np.load("ransac.npy", allow_pickle=True).item()
points = ransac["points"]
x1 = ransac["x1"]
x2 = ransac["x2"]
points.shape, x1.shape, x2.shape

((2, 100), (2,), (2,))

In [ ]:
# Ex 7.6
# Function to fit a line, in homo coords, given 2 points


def fit_line(p1, p2):
    """
    Fits a line given 2 points.

    Args:
        p1, p2 (np.array) : 2x1 inhomogenous coordinates

    Returns:
        l : 3x1, line in homogenous coordinates
    """
    if p1.shape == (2,):
        p1 = p1.reshape(2, 1)
        p2 = p2.reshape(2, 1)
    if p1.shape != (2, 1) or p2.shape != (2, 1):
        raise ValueError("Points must be 2x1 np.array")

    p1h = Piinv(p1)
    p2h = Piinv(p2)
    # cross() requires input as vectors
    l = np.cross(p1h.squeeze(), p2h.squeeze())
    return l

eqn of line: [-3.  1.  5.]


You are fitting a straight line to a set of 2D points (points) with RANSAC.
In the current iteration you fit the line through x1 and x2.
tau = 0.2
![qn1](images/Q1.png)

In [18]:
def dist_to_line(pt: np.array, l: np.array):
    """
    Obtains the shortest(perpendicular) distance between a point and a line
        Args:
            pt (np.array): point
            l (np.array): line, [a, b, c], shape (3,)

        Return:
            dist (float): distance between point and line
    """
    if pt.shape[0] == 2:
        pt = Piinv(pt)
    pt = np.squeeze(pt.T)
    dist = np.abs(np.dot(l, pt.T)) / (abs(pt[2]) * np.sqrt(l[0] ** 2 + l[1] ** 2))
    return dist

In [19]:
l = fit_line(x1, x2)
l, Piinv(points).shape

(array([ 0.13825614,  0.10280998, -0.16058676]), (3, 100))

In [20]:
# Ex 1.7
def point_line_distance(line: np.ndarray, p: np.ndarray):
    """
    Calculate shortest distance d between line l and 2D homogenous point p.

    Args:
        line: homogenous line, shape (3, 1)
        p: 3x1 vector, shape (3, 1)

    Returns:
        d (float): distance
    """
    if p.shape != (3, 1):
        raise ValueError("p must be a 3x1 homogenous vector")
    if line.shape != (3, 1):
        raise ValueError("line must be a 3x1 homogenous vector")

    d = abs(line.T @ p) / (abs(p[2]) * np.sqrt(line[0] ** 2 + line[1] ** 2))
    return d

In [21]:
def summarise_points(pts: np.array, l: np.array, threshold: float):
    """
    Determines a boolean set of 2D points of inliers or outliers with respect
    to a given line. True represents inliners and False represents outliers
        Args:
            pts (np.array): original set of pts first point in the lin determines which of a set of 2D points are an inliers or outliers with respect
            to a given line. Expected shape (3 x N). Expect homogeneous points.
            l (np.array): line
            threshold (np.array): threshold distance between line l and pts, used to separate inliers and outliers

        Return:
            inliers (np.array): array with all the inliers
            outliers (np.array): array with all the outliers
    """
    inliers = []
    outliers = []
    num_pts = pts.shape[1]
    for i in range(num_pts):
        pt = pts[:, i]
        if pt.shape[0] == 3:
            pt = pt.reshape((3, 1))
        if pt.shape[0] == 2:
            pt = Piinv(pt.reshape((2, 1)))
        distance = dist_to_line(pt, l)
        if distance <= threshold:
            inliers.append(pt)
        else:
            outliers.append(pt)
    inliers = np.array(inliers)
    outliers = np.array(outliers)
    return inliers, outliers

In [ ]:
# Ex 7.7
# Function to determine inliners and outliers


def find_inliners_outliers(l, points, tau):
    """
    Args:
        l : equation of line in homogenous coordinates
        tau : threshold for inliners

    Returns:
        inliners (np.array) : 2xa, set of inliner points
        outliers (np.array) : 2xb, set of outlier points
    """
    inliners = []
    outliers = []
    for p in points.T:
        p = p.reshape(2, 1)
        ph = Piinv(p)
        d = abs(l.T @ ph) / (abs(ph[2]) * np.sqrt(l[0] ** 2 + l[1] ** 2))
        if d <= tau:  # inlier
            inliners.append(p)
        else:  # outlier
            outliers.append(p)
    inliners = np.array(inliners).squeeze().T
    outliers = np.array(outliers).squeeze().T
    return inliners, outliers


inliners, outliers = find_inliners_outliers(l, points, 0.2)

print(f"inliners.shape: {inliners.shape}, outliers.shape: {outliers.shape}")

inliners.shape: (2, 34), outliers.shape: (2, 66)


In [23]:
inliers, outliers = summarise_points(Piinv(points), l, 0.2)
inliers.shape, outliers.shape

((34, 3, 1), (66, 3, 1))

We are using RANSAC to estimate a homography matrix.
At iteration number 191 we find a model where 103 out of 404
point matches are inliers, which is the highest number of inliers
we have observed so far.

Given the current information, what is the smallest number of iterations we need to run in total in order to be 95% sure that we will have fitted at least one model to only inliers?
![Qn2](images/Q2.png)

In [25]:
p = 0.95
estimate_outlier_prob = 1 - 103 / 404
num_iterations = np.log(1 - p) / np.log(1 - ((1 - estimate_outlier_prob) ** 4))
num_iterations

np.float64(707.5554825030302)